## Some test with k means cluster

Currently not in use

In [ ]:
Difference plot for capacities in the NUTS regions 

In [ ]:
#Join the NUTS regions with the coordinates of the wind turbines
H_Wind_and_NUTS = gpd.sjoin(H_Wind, NUTS, op='within')
    
thewindpower_and_NUTS = gpd.sjoin(thewindpower, NUTS, op='within')
    
#Group the data per NUTS regions
H_Wind_and_NUTS_agg = H_Wind_and_NUTS[['capacity','NUTS_ID','country']].groupby(['NUTS_ID'])\
                                                        .agg({'capacity' : ['sum','mean', 'count', 'size']})
    
thewindpower_and_NUTS_agg =thewindpower_and_NUTS[['capacity','NUTS_ID','country']].groupby(['NUTS_ID'])\
                                                        .agg({'capacity' : ['sum','mean', 'count', 'size']})
    
#Join data into one dataframe
Data_join = NUTS.join(H_Wind_and_NUTS_agg['capacity']['sum'], on='NUTS_ID', how='inner').rename(columns={"sum": "H_Wind_Cap"})
    
Data_join = Data_join.join(thewindpower_and_NUTS_agg['capacity']['sum'], on='NUTS_ID', how='inner').rename(columns={"sum": "thewindpower_Cap"})
    
#area of NUTS regions (CRS need to be changed because crs is not in meter)
Data_join["area"] = Data_join['geometry'].to_crs(epsg='3395')\
                                             .map(lambda p: p.area / 10**6)
    
    
Data_join['thewindpower_Cap_Density'] = Data_join.thewindpower_Cap / Data_join.area
Data_join['H_Wind_Cap_Density'] = Data_join.H_Wind_Cap / Data_join.area

Data_join['Density_dif'] = Data_join['H_Wind_Cap_Density'] - Data_join['thewindpower_Cap_Density']

In [ ]:
# new dataframe for capacity per country
Data_NUTS = pd.DataFrame()

Data_NUTS = Data_join[['H_Wind_Cap','thewindpower_Cap', 'NUTS_ID']].copy()
Data_NUTS.set_index('NUTS_ID', inplace=True)

# Capacity difference between NUTS regions  
Data_NUTS['dif'] = Data_NUTS['H_Wind_Cap'] - Data_NUTS['thewindpower_Cap']
Data_NUTS.plot.bar(title='Capacity difference between NUTS regions')

In [ ]:
Currently not in use

X_H_Wind.power.sum()

K_clusters = range(1,10)

kmeans = [KMeans(n_clusters=i) for i in K_clusters]

Y_axis = X_H_Wind[['Lat']]

X_axis = X_H_Wind[['Lon']]

score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize

plt.plot(K_clusters, score)

plt.xlabel('Number of Clusters') plt.ylabel('Score') plt.title('Elbow Curve') plt.show()

kmeans = KMeans(n_clusters = 10, init ='k-means++')

kmeans.fit(X_H_Wind[X_H_Wind.columns[1:3]]) # Compute k-means clustering.

X_H_Wind['cluster_label'] = kmeans.fit_predict(X_H_Wind[X_H_Wind.columns[1:3]])

centers = kmeans.cluster_centers_ # Coordinates of cluster centers.

labels = kmeans.predict(X_H_Wind[X_H_Wind.columns[1:3]]) # Labels of each point

X_H_Wind.head(10)

X_H_Wind.plot.scatter(x = 'Lat', y = 'Lon', c=labels, s=50, cmap='viridis')

plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

thewindpower

X_thwindpower=thewindpower.loc[thewindpower.ISO_code == 'DE',['Total_power','Latitude','Longitude']]

kmeans = KMeans(n_clusters = 10, init ='k-means++')

kmeans.fit(X_thwindpower[X_thwindpower.columns[1:3]]) # Compute k-means clustering.

X_thwindpower['cluster_label'] = kmeans.fit_predict(X_thwindpower[X_thwindpower.columns[1:3]])

centers_2 = kmeans.cluster_centers_ # Coordinates of cluster centers.

labels_2 = kmeans.predict(X_thwindpower[X_thwindpower.columns[1:3]]) # Labels of each point

X_thwindpower.head(10)

X_thwindpower.plot.scatter(x = 'Latitude', y = 'Longitude', c=labels_2, s=50, cmap='viridis')

plt.scatter(centers_2[:, 0], centers_2[:, 1], c='black', s=200, alpha=0.5)

def get_zipcode(i, geolocator, lat_field, lon_field): try: location = geolocator.reverse((i[lat_field], i[lon_field])) return location.raw['address']['postcode'] except: return np.nan

geolocator = geopy.Nominatim(user_agent='my-application')

#example df = pd.DataFrame({ 'Lat': [29.39291, 29.39923, 29.40147, 29.38752, 29.39291, 29.39537, 29.39343, 29.39291, 29.39556], 'Lon': [-98.50925, -98.51256, -98.51123, -98.52372, -98.50925, -98.50402, -98.49707, -98.50925, -98.53148] }) zipcodes = df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Lat', lon_field='Lon')

thewindpower['zipcode'] = thewindpower.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')

H_Wind['zipcode'] = H_Wind.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Lat', lon_field='Lon')

thewindpower.ISO_code.unique()

thewindpower_DE = thewindpower[thewindpower.ISO_code =='DE'] thewindpower_DE['zipcode'].isna().sum()

H_Wind_DE = H_Wind[H_Wind.country == 'DE'] H_Wind_DE['zipcode'].isna().sum()

H_Wind_DE_agg = H_Wind_DE[['turbines','power','country','X','Y','zipcode']].groupby(['zipcode'])
.agg({
'turbines': ['sum','mean', 'count', 'size'], 'power' : ['sum','mean', 'count', 'size'], 'X' : ['count', 'size'], 'Y' : ['count', 'size']}) Data_1 = H_Wind_DE_agg['power']['sum'] Data_1 = Data_1.to_frame() Data_1.rename(columns={"sum": "H_Wind"}, inplace=True)

thewindpower_DE_agg =thewindpower_DE[['Hub_height', 'Number_of_turbines', 'Total_power', 'ISO_code', 'Latitude', 'Longitude', 'zipcode']].groupby(['zipcode'])
.agg(['sum','mean', 'count', 'size'
]) Data_2 = thewindpower_DE_agg['Total_power']['sum'] Data_2 = Data_2.to_frame() Data_2.rename(columns={"sum": "the_wind_power"}, inplace=True)

Combine the two datasets
